In [1]:
#Mount Drive and install dependencies if running in Colab
def install_dependecies():
  !pip install transformers
  !pip install pytorch-lightning

from sys import path
import os
import sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  root_PATH = '/content/drive/My Drive/nlp-seminar/repository'
  drive_mount_location = '/content/drive'
  module_path = root_PATH + '/src'
  
  drive.mount(drive_mount_location, force_remount=True)
  path.append(root_PATH)

  install_dependecies()
else:
  root_PATH = os.path.abspath("../../..")
  module_path = os.path.abspath(os.path.join('../../../src'))

%load_ext autoreload
%autoreload 2

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from data_processor import DataProcessor
from model_evaluator import ModelEvaluator
from custom_dataset import CustomDataset
from models.bert_custom_model import BERTCustomModel

import pandas as pd
from torch import cuda
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import transformers
import numpy as np
from sklearn import metrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import accuracy_score
import pytorch_lightning as pl
from pytorch_lightning import Trainer

/home/prestrepoh/anaconda3/envs/idp2020/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
/home/prestrepoh/anaconda3/envs/idp2020/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,


In [3]:
# Script configuration
MAX_LEN = 200
TEST_BATCH_SIZE = 32

model_to_use = 'bert-base-uncased'
model_path = root_PATH + '/notebooks/deep_learning_methods/BERT/tb_logs/my_BERT_model/version_0/checkpoints/epoch=1.ckpt'

#Model parameters
model_params = {
                'test_batch_size': TEST_BATCH_SIZE
                }

gpus_to_use = [4,5,6,7]

/home/prestrepoh/anaconda3/envs/idp2020/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#Load datasets
test_df = pd.read_csv(root_PATH + '/data/test.csv')

#Get boolean mask of the datasets
test_boolean_mask = test_df.iloc[:,8:]

#Get topics present in the dataset
remaining_topics = test_boolean_mask.columns.tolist()

#Create new column with the boolean mask
test_df["list"] = test_boolean_mask.values.tolist()

#Remove columns not necessary from the datasets
test_df = test_df[["conversation","list"]].reset_index()

In [5]:
tokenizer = BertTokenizer.from_pretrained(model_to_use)

testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)
data_loader = DataLoader(testing_set, batch_size=TEST_BATCH_SIZE, shuffle= False, num_workers=32)

In [6]:
model = BERTCustomModel.load_from_checkpoint(
    model_path, 
    hparams = model_params, 
    training_dataset=None, 
    validation_dataset=None, 
    labels=remaining_topics, 
    model_to_use=model_to_use
    )

trainer = pl.Trainer(gpus=gpus_to_use,
                             accelerator='dp',
                             #limit_train_batches=32,
                             #limit_val_batches=32,
                             #limit_test_batches=60,
                             #max_epochs=200,
                             #logger=logger,
                             #default_root_dir=logs_path
                             )

trainer.test(ckpt_path=model_path,model=model,test_dataloaders=[data_loader])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4,5,6,7]


Total accuracy: 0.8756819382642961
Accuracy per label: {'Satisfied users': 0.9383534174894487, 'Bugs': 0.9874975332454851, 'Design & UX': 0.9919026793728396, 'Dissatisfied users': 0.9757335013909313, 'Performance': 0.9950537593338807, 'Use cases': 0.9948627847907874, 'Gaming': 0.9958176575062544, 'Feature Requests': 0.9940415942554858, 'Complexity': 0.9961486800476164, 'Pricing': 0.9957094385985015, 'Security & Accounts': 0.9984594720190465, 'Update': 0.9967661644036183, 'Camera & Photos': 0.9985040327457684, 'Video': 0.9993888814621011, 'Customer Support': 0.9970971869449803, 'Notifications & Alerts': 0.9990005665578112, 'Frequency': 0.9981857418406126, 'Advertising': 0.9990514931026361, 'Payment': 0.9986568123802431, 'Connectivity': 0.9985358618362838, 'Devices': 0.9979629382070037, 'Audio': 0.9992297360095233, 'Sign Up & Login': 0.9989560058310893, 'Location Services': 0.9997135381853599, 'Privacy': 0.9993188574629669, 'Internationalization': 0.9996944407310505, 'no topic': 0.879418

1